In [1]:
import numpy as np
from abc import ABC, abstractmethod
import overpy
import pandas as pd

In [2]:
states = {
    'waiting': 0,
    'assigned': 1,
    'riding': 2,
    'arrived': 3
}


class Person:
    
    def __init__(self, s, t, time_init):
        self.s = s
        self.t = t
        self.state = states['waiting']
        self.time_init = time_init
    
    def start_ride(self, time_start):
        self.state = states['riding']
        self.time_start = time_start
    
    def finish_ride(self, time_finish):
        self.state = states['arrived']
        self.time_finish = time_finish
    
    def stats(self):
        return {
            'waiting': self.time_start - self.time_init,
            'ride': self.time_finish - self.time_start
        }
    
    def __repr__(self):
        return f'Person from {self.s} to {self.t}, appeared at {self.time_init}'


class PersonFactory:
    
    def generate_person(self, nodes, time):
        return Person(*np.random.choice(nodes, 2, replace=False), time)

    
class Car:
    
    def __init__(self, start_node, avg_speed, time_init, limit):
        self.at = start_node
        self.avg_speed = avg_speed
        self.time_init = time_init
        self.limit = limit
        self.passengers = dict(zip(range(limit), [None] * limit))
    
    def start_ride(self, dest, busy_until):
        self.at = dest
        self.busy_until = busy_until
    
    def is_full(self):
        return all(v != None for v in self.passengers.values())
    
    def pick_up_passenger(self, passenger: Person):
        i = 0
        while self.passengers[i] is not None:
            i += 1
        self.passengers[i] = passenger


class CarFactory:
    
    def generate_car(self, nodes, time):
        return Car(
            np.random.choice(nodes),
            np.random.uniform(20, 40),
            time,
            4
        )


class Policy(ABC):
    
    def __init__(self, cars: list, people: list):
        self.cars = cars
        self.people = people
    
    @abstractmethod
    def assign(self, car_id: int, time: int):
        pass
    
    @abstractmethod
    def step(self, time: int):
        pass


class DummyPolicy(Policy):
    
    def assign(self, car_id: int, time: int):
        pass
    
    def step(self, time: int):
        pass

In [3]:
class Map:
    
    def __init__(self, S: float, W: float, N: float, E: float):
        # TODO: make it prettier
        self.area = overpy.Overpass().query(
            '''(way["highway"](%s););
                (._;>;);
                out;''' 
            % (', '.join(['%.10f' % x for x in [S, W, N, E]])))
    
    def get_map_nodes(self):
        node_cols = ['id','lon','lat']
        return pd.DataFrame([[getattr(node,a) for a in node_cols] for node in self.area.nodes])\
            .rename(columns=dict(zip(range(len(node_cols)),node_cols)))


In [4]:
city = Map(58.3691, 26.6906, 58.3892, 26.7549)

nodes = city.get_map_nodes()

In [5]:
time = 0
car_proba  = 0.03
pers_proba = 0.15

car_f = CarFactory()
per_f = PersonFactory()

cars = [car_f.generate_car(nodes.index, time)]
people = [per_f.generate_person(nodes.index, time), per_f.generate_person(nodes.index, time)]

while time <= 10 ** 4:
    if np.random.rand() < car_proba:
        cars.append(car_f.generate_car(nodes.index, time))
    if np.random.rand() < pers_proba:
        people.append(per_f.generate_person(nodes.index, time))
    
    time += 1
